## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-07-07-09-28-51.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['June']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    1128
Name: count, dtype: int64
-------
$/SQUARE FEET
False    1109
True       19
Name: count, dtype: int64
-------
YEAR BUILT
False    1128
Name: count, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
699,PAST SALE,June-29-2023,Condo/Co-op,8741 Wiles Rd #103,Coral Springs,FL,33067.0,300.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Springs/8741-W...,MARMLS,A11375277,N,Y,26.287879,-80.244946
416,PAST SALE,June-12-2023,Condo/Co-op,9430 Live Oak Pl #304,Davie,FL,33324.0,20000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Davie/9430-Live-Oak-...,Beaches MLS,F10378228,N,Y,26.095385,-80.278083
953,PAST SALE,June-23-2023,Condo/Co-op,5100 SW 41st St #209,Pembroke Park,FL,33023.0,60000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Park/5100-S...,Beaches MLS,F10362924,N,Y,25.973443,-80.193348
235,PAST SALE,June-2-2023,Condo/Co-op,6650 Royal Palm Blvd Unit 305C,Margate,FL,33063.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/6650-Royal-P...,Beaches MLS,F10376937,N,Y,26.252239,-80.214245
57,PAST SALE,June-20-2023,Condo/Co-op,517 Durham R #517,Deerfield Beach,FL,33442.0,79000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/517-...,Beaches MLS,F10374703,N,Y,26.317285,-80.138796
515,PAST SALE,June-5-2023,Condo/Co-op,2301 NW 41st Ave #202,Lauderhill,FL,33313.0,79000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/2301-NW-4...,Beaches MLS,F10368707,N,Y,26.155719,-80.204461
1074,PAST SALE,June-26-2023,Condo/Co-op,9661 Sunrise Lakes Blvd #207,Sunrise,FL,33322.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9661-Sunrise...,MARMLS,A11294465,N,Y,26.164064,-80.278409
436,PAST SALE,June-30-2023,Condo/Co-op,8080 N Sunrise Lakes Dr #209,Sunrise,FL,33322.0,80000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/8080-N-Sunri...,Beaches MLS,F10381567,N,Y,26.162691,-80.254479
650,PAST SALE,June-20-2023,Condo/Co-op,8510 Sunrise Lakes Blvd #308,Sunrise,FL,33322.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/8510-Sunrise...,Beaches MLS,F10376006,N,Y,26.163673,-80.266238
926,PAST SALE,June-28-2023,Condo/Co-op,2800 Somerset Dr Unit 405J,Lauderdale Lakes,FL,33311.0,83000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/280...,Beaches MLS,F10365270,N,Y,26.160830,-80.188993


In [13]:
print(df_filtered['URL'].iloc[699])

https://www.redfin.com/FL/Coral-Springs/8741-Wiles-Rd-33067/unit-103/home/42050154


In [38]:
# Correct the prices, if needed
df_filtered.at[699,'PRICE']=(300000)
df_filtered.at[416,'PRICE']=(200000)

In [17]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [14]:
# # Corrections, if needed
df_filtered.at[699,'$/SQUARE FEET']=(300000/984)
df_filtered.at[416,'$/SQUARE FEET']=(200000/730)

In [15]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
953,60000.0,5100 SW 41st St #209,Pembroke Park,82.0
436,80000.0,8080 N Sunrise Lakes Dr #209,Sunrise,94.0
771,100000.0,2860 Somerset Dr Unit 315K,Lauderdale Lakes,99.0
27,118000.0,671 S Hollybrook Dr #301,Pembroke Pines,101.0
515,79000.0,2301 NW 41st Ave #202,Lauderhill,105.0
536,91500.0,6201 N Falls Cir Dr #409,Lauderhill,105.0
920,93000.0,6001 N Falls Circle Dr #206,Lauderhill,107.0
828,95000.0,5901 NW 61st Ave #209,Tamarac,108.0
1077,112000.0,9481 Sunrise Lakes Blvd #302,Sunrise,109.0
161,128000.0,8260 SW 24th St #6311,North Lauderdale,110.0


In [16]:
print(df_filtered.URL.iloc[953])

https://www.redfin.com/FL/Pembroke-Park/5100-SW-41st-St-33023/unit-209/home/42007118


In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [21]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [22]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [23]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [24]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [25]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [26]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"June 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [27]:
m.save('index.html')

## Summary Info

In [28]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [29]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [30]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [48]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Mangonia-Park/910-E-Tiffany-Dr-33407/unit-2/home/42431003


In [43]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
4001 North Ocean Condo, 4001 N Ocean Blvd #504 Gulf Stream | Price $8,000,000 | $2,449 psf | Year built: 2013
Least Expensive
Tiffany Lake Condo PH 1, 910 E Tiffany Dr #2 Mangonia Park | Price $62,258 | $64 psf | Year built: 1973
Highest Price Per Square Foot
4001 North Ocean Condo, 4001 N Ocean Blvd #504 Gulf Stream | Price $8,000,000 | $2,449 psf | Year built: 2013
Lowest Price Per Square Foot
Tiffany Lake Condo PH 1, 910 E Tiffany Dr #2 Mangonia Park | Price $62,258 | $64 psf | Year built: 1973
Newest
La Clara, 200 Arkona Ct #804 West Palm Beach | Price $3,979,920 | $1,286 psf | Year built: 2023
Oldest
Palm Beach Hotel Condo, 235 Sunrise Ave #3201 Palm Beach | Price $550,000 | $1,146 psf | Year built: 1925


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [45]:
from datetime import datetime, timedelta

date1 = datetime(2023, 6, 8) ## List (Earlier) date
date2 = datetime(2023, 6, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

4


## Map URL Snagger

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [34]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PB_condo_sales_month_ending_june_2023


## Get Summary Data

In [49]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 822
--------
Total sale price: $397,562,783
--------
Median sale price: $280,000
--------
Max sale price: $8,000,000
--------
Min sale price: $62,258
------------------------------------------------
PSF INFO
Max price per square foot: $2,449
--------
Min price per square foot: $64
--------
Median price per square foot: $238
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1925.0
----------
Average building age: 1983.9282238442822
